In [1]:
import gradio as gr
import numpy as np
import pandas as pd
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image
import psycopg2

In [2]:
# pipe_A = StableDiffusionPipeline.from_pretrained(
#     "CompVis/stable-diffusion-v1-4", 
#     torch_dtype=torch.float32)

# pipe_B = StableDiffusionPipeline.from_pretrained(
#     "runwayml/stable-diffusion-v1-5", 
#     torch_dtype=torch.float32)

In [5]:
def infer(prompt, steps): 
    
    # this is the actual use of the huggingface pipe
    # for now, comment this out, and instead
    # load test-images directly
    
#     scale = 25
#     seed = 123456789
#     generator = torch.Generator(device=device).manual_seed(seed)
#     images_list_A = pipe_A(
#         [prompt] * samples,
#         num_inference_steps=steps,
#         guidance_scale=scale,
#         generator=generator,
#     )
#     images_list_B = pipe_B(
#         [prompt] * samples,
#         num_inference_steps=steps,
#         guidance_scale=scale,
#         generator=generator,
#     )
#     images_A, images_B = [], []
#     for i, image in enumerate(images_list_A["sample"]):
#         image.save(f"gradio_outputs/model_A/{prompt}_{i}.png")
#         images_A.append(image)
#     for i, image in enumerate(images_list_B["sample"]):
#         image.save(f"gradio_outputs/model_B/{prompt}_{i}.png")
#         images_B.append(image)
#     return images_A + images_B

    # load test images directly instead of using hugginface pipe
    image_list_A = [Image.open("sample_img_A.png")]*samples
    image_list_B = [Image.open("sample_img_B.png")]*samples
    for i, image in enumerate(image_list_A):
        image.save(f"gradio_outputs/model_A/{prompt}_{i}.png")
    for i, image in enumerate(image_list_B):
        image.save(f"gradio_outputs/model_B/{prompt}_{i}.png")
    return image_list_A + image_list_B


def give_feedback(prompt, better_model):
    print(f"For {prompt}, you preferred {better_model}")
    
#     connection = psycopg2.connect(user="postgresGradiff",
#                                   password="DatathonGradioDiffusion2022",
#                                   host="http://database-ak-1.cjpo4rqs9rfq.us-east-2.rds.amazonaws.com/",
#                                   port="8080",
#                                   database="postgres_db")
#     cursor = connection.cursor()

#     postgres_insert_query = """ INSERT INTO diffusion_compare (PROMPT, WINNER) VALUES (%s,%s)"""
#     record_to_insert = (prompt, better_model)
#     cursor.execute(postgres_insert_query, record_to_insert)

#     connection.commit()

samples = 3
device = "cpu"
block = gr.Blocks()

with block:
    with gr.Group():
        with gr.Box():
            with gr.Row().style(equal_height=True):
                text = gr.Textbox(
                    label="Enter your prompt",
                    show_label=False,
                    max_lines=1,
                    placeholder="Enter your prompt",
                )
                btn = gr.Button("Generate images")
                
        gallery = gr.Gallery(
            label="Generated images", show_label=False, elem_id="gallery"
        ).style(grid=[6], height="auto")

        with gr.Row(elem_id="options"):
            steps = gr.Slider(label="Steps", minimum=1, maximum=50, value=2, step=1)
        text.submit(infer, inputs=[text, steps], outputs=gallery)
        btn.click(infer, inputs=[text, steps], outputs=gallery)
        with gr.Box():
            with gr.Row().style(equal_height=True):
                feedback_text = gr.Textbox(
                    label="Give feedback",
                    show_label=False,
                    max_lines=2,
                    placeholder="Which model did better, A or B?",
                )
                feedback_btn = gr.Button("Submit feedback")
        feedback_text.submit(give_feedback, inputs=[text, feedback_text], outputs=gallery)
        feedback_btn.click(give_feedback, inputs=[text, feedback_text], outputs=gallery)
        
block.launch()

Running on local URL:  http://127.0.0.1:7883

To create a public link, set `share=True` in `launch()`.


For asdf, you preferred A
For apple, you preferred B
